In [17]:
import requests
import re

## Create day value for query

In [18]:
day = []
for i in range(1,13):
    if i < 10:
        date = '2018-0' + str(i) + '-01'
    else:
        date = '2018-' + str(i) + '-01'
    day.append(date)
    
for i in range(1,13):
    if i < 10:
        date = '2019-0' + str(i) + '-01'
    else:
        date = '2019-' + str(i) + '-01'
    day.append(date)


num = len(day) - 1

### Sample query
select distinct(partner_code) from :parquetTppTransDailyHis: where trans_amount > 0 and service_code in ('EVN','NHCM','NDT ','FECRDT','HCDEBT','PCE','100000','000003 ','000004','200000','PAYMENT','CP','VNPPRE','VMSPRE','BEEPRE','VNMPRE','VNPPCE','VMSPCE','BEEPCE','VNMPCE','DNIWACO ','NHUE ','TGGWACO','CTWACO','PRUDENTIAL','OCBCRDT','MCREDIT','MSBCRDT','NCM','HCDEBT','VAP','MIRAEASSET','ACSCRDT	','ATMCRDT','THBVAG','GATE','ONCASH','MEGACARD','ZINGPCE','GARENA','VCOIN','BTAINS','SSTADSBILL','VEXERE','STCBILLING','PRUFINANCE','KPLUS','AVG','VTVCAB') and error_code in ('00','0') and request_day <'2019-01-01' and request_day<'2019-02-01'

## Create query

In [8]:
reqs = []
for i in range(num):
    req = "select distinct(partner_code) from :parquetTppTransDailyHis: where trans_amount > 0 and service_code in ('EVN','NHCM','NDT ','FECRDT','HCDEBT','PCE','100000','000003 ','000004','200000','PAYMENT','CP','VNPPRE','VMSPRE','BEEPRE','VNMPRE','VNPPCE','VMSPCE','BEEPCE','VNMPCE','DNIWACO ','NHUE ','TGGWACO','CTWACO','PRUDENTIAL','OCBCRDT','MCREDIT','MSBCRDT','NCM','HCDEBT','VAP','MIRAEASSET','ACSCRDT	','ATMCRDT','THBVAG','GATE','ONCASH','MEGACARD','ZINGPCE','GARENA','VCOIN','BTAINS','SSTADSBILL','VEXERE','STCBILLING','PRUFINANCE','KPLUS','AVG','VTVCAB') and error_code in ('00','0') and request_day <'" + day[i+1] + "' and request_day> '" + day[i] + "'"
    reqs.append(req)

In [10]:
reqs

["select distinct(partner_code) from :parquetTppTransDailyHis: where trans_amount > 0 and service_code in ('EVN','NHCM','NDT ','FECRDT','HCDEBT','PCE','100000','000003 ','000004','200000','PAYMENT','CP','VNPPRE','VMSPRE','BEEPRE','VNMPRE','VNPPCE','VMSPCE','BEEPCE','VNMPCE','DNIWACO ','NHUE ','TGGWACO','CTWACO','PRUDENTIAL','OCBCRDT','MCREDIT','MSBCRDT','NCM','HCDEBT','VAP','MIRAEASSET','ACSCRDT\t','ATMCRDT','THBVAG','GATE','ONCASH','MEGACARD','ZINGPCE','GARENA','VCOIN','BTAINS','SSTADSBILL','VEXERE','STCBILLING','PRUFINANCE','KPLUS','AVG','VTVCAB') and error_code in ('00','0') and request_day <'2018-02-01' and request_day> '2018-01-01'",
 "select distinct(partner_code) from :parquetTppTransDailyHis: where trans_amount > 0 and service_code in ('EVN','NHCM','NDT ','FECRDT','HCDEBT','PCE','100000','000003 ','000004','200000','PAYMENT','CP','VNPPRE','VMSPRE','BEEPRE','VNMPRE','VNPPCE','VMSPCE','BEEPCE','VNMPCE','DNIWACO ','NHUE ','TGGWACO','CTWACO','PRUDENTIAL','OCBCRDT','MCREDIT','MSBCRD

## Define function to send request to API

In [22]:
def post(req):
    API_ENDPOINT = "http://10.58.244.171:9126/job"
    
    data = {'query':req, 
        'JobName':'test', 
        'private': 'true'} 
    r = requests.post(url = API_ENDPOINT, data = data)
    return r.text

### Example of sending query with function
pastebin_url = post('''select distinct(shop_code) from :parquetTppTransDailyHis: where trans_amount > 0 and service_code in ('EVN','NHCM','NDT ','FECRDT','HCDEBT','PCE','100000','000003 ','000004','200000','PAYMENT','CP','VNPPRE','VMSPRE','BEEPRE','VNMPRE','VNPPCE','VMSPCE','BEEPCE','VNMPCE','DNIWACO ','NHUE ','TGGWACO','CTWACO','PRUDENTIAL','OCBCRDT','MCREDIT','MSBCRDT','NCM','HCDEBT','VAP','MIRAEASSET','ACSCRDT	','ATMCRDT','THBVAG','GATE','ONCASH','MEGACARD','ZINGPCE','GARENA','VCOIN','BTAINS','SSTADSBILL','VEXERE','STCBILLING','PRUFINANCE','KPLUS','AVG','VTVCAB') and error_code in ('00','0') and request_day <'2019-01-01' and request_day<'2019-02-01''')

## Sending multiple POST request to API with predefined queries

In [19]:
job = []
for i in range(num):
    job.append(post(reqs[i]))

## Take the job ID to download data from API

In [20]:
jobid = []
for i in range(num):
    jobid.extend(re.findall(r"\d+",job[i]))

## Define function to download multiple data file to a target folder

In [24]:
def download(jobid):
    from bs4 import BeautifulSoup
    import urllib
    import requests
    
    URL = 'http://10.58.244.171:9126/job?jobId=' + jobid +'&private=true' 
    soup = BeautifulSoup(urllib.request.urlopen(URL))

    url = re.findall("http.+csv",soup.text)[0]
    
    myfile = requests.get(url, allow_redirects=True)
     
    folder = 'D:/Data/Data Export/San San/Untitled Folder/Data/' + str(i) + '.csv'
    
#     open(folder, 'wb').write(myfile.content)
    
    return open(folder, 'wb').write(myfile.content)

## Download data

In [31]:
for i in range (num):
    download(jobid[i])